In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics
import random
from random import randint
import time
import os
import numpy as np
from collections import namedtuple
import cv2
from pathlib import Path
import argparse
import os
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow import keras

# pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

In [2]:
### 실행결과 동일하게(완전 일치하지는 않음)
tf.keras.utils.set_random_seed(42)
### 텐서 연산 고정(완전 일치하지는 않음)
tf.config.experimental.enable_op_determinism()

In [3]:
# data = 원본데이터 | data2 = 원본데이터 복붙 | data3 = 우리가 측정한 데이터
folder_path_1 = "./data/이상(방화)"
folder_path_2 = "./data/이상(유기)"
folder_path_3 = "./data/이상(전도)"
folder_path_5 = "./data/가방"
folder_path_6 = "./data/주머니"
folder_path_7 = "./data/이상(파손)"
folder_path_8 = "./data/이상(폭행)"
folder_path_9 = "./data/이상(흡연)"
folder_path_10 = "./data/구매(구매)"
folder_path_11 = "./data/구매(반품)"
folder_path_12 = "./data/구매(비교)"
folder_path_13 = "./data/구매(선택)"

In [4]:
def concat(path):
    file_lst = os.listdir(path)
    df_list = []
    for file in file_lst:
        file_name = os.path.join(path, file)
        df = pd.read_csv(file_name)
        df = df[df.NUMOFBODIES != 0].reset_index(drop=True)
###
        if len(df) < 610:
            pad_length = 610 - len(df)
            pad_df = pd.DataFrame(0, index=range(len(df), 610), columns=df.columns)
            df = pd.concat([df, pad_df])
        df['index_num'] = df.index
###
        df_list.append(df)
    #concat_df = pd.concat(df_list, ignore_index=True)
    return df_list

In [5]:
file_lst = os.listdir(folder_path_1)

In [6]:
fire_mer = concat(folder_path_1)
yugi_mer = concat(folder_path_2)
jeon_mer = concat(folder_path_3)
#move_mer = concat(folder_path_4)
bag_mer = concat(folder_path_5)
pocket_mer = concat(folder_path_6)
damage_mer = concat(folder_path_7)
violence_mer = concat(folder_path_8)
smoke_mer = concat(folder_path_9)
buy_mer = concat(folder_path_10)
refund_mer = concat(folder_path_11)
compar_mer = concat(folder_path_12)
select_mer = concat(folder_path_13)


In [7]:
buy_mer = pd.concat(buy_mer, ignore_index=True)
refund_mer = pd.concat(refund_mer, ignore_index=True)
compar_mer = pd.concat(compar_mer, ignore_index=True)
select_mer = pd.concat(select_mer, ignore_index=True)
#move_mer = pd.concat(move_mer, ignore_index=True)
fire_mer = pd.concat(fire_mer, ignore_index=True)
yugi_mer = pd.concat(yugi_mer, ignore_index=True)
jeon_mer = pd.concat(jeon_mer, ignore_index=True)
bag_mer = pd.concat(bag_mer, ignore_index=True)
pocket_mer = pd.concat(pocket_mer, ignore_index=True)
damage_mer = pd.concat(damage_mer, ignore_index=True)
violence_mer = pd.concat(violence_mer, ignore_index=True)
smoke_mer = pd.concat(smoke_mer, ignore_index=True)

In [8]:
# 예측할 행위를 1, 나머지를 0으로 라벨링해서 11개 모델 다 따로 생성하기
# 정상행동 = 0, 이상행동 = 1
buy_mer['LABEL'] = 0       #0
refund_mer['LABEL'] = 0    #1
compar_mer['LABEL'] = 0    #2
select_mer['LABEL'] = 0    #3
#move_mer['LABEL'] = 0      #4
fire_mer['LABEL'] = 0      #5
yugi_mer['LABEL'] = 0      #6
jeon_mer['LABEL'] = 0      #7
bag_mer['LABEL'] = 0       #8
pocket_mer['LABEL'] = 0    #9
damage_mer['LABEL'] = 0    #10
violence_mer['LABEL'] = 0  #11
smoke_mer['LABEL'] = 1     #12

In [9]:
mer_lst = [fire_mer,
           yugi_mer,
           jeon_mer,
           bag_mer,
           pocket_mer,
           damage_mer,
           violence_mer,
           smoke_mer,
           #move_mer,
           buy_mer,
           refund_mer,
           compar_mer,
           select_mer]
full_data = pd.concat(mer_lst, ignore_index=True)
full_data.shape

(2821860, 91)

In [10]:
full_data.head(170)

,NUMOFBODIES,ID,PELVIS_X,PELVIS_Y,PELVIS_CONFIDENCE_LEVEL,SPINE_NAVAL_X,SPINE_NAVAL_Y,SPINE_NAVA_CONFIDENCE_LEVEL,SPINE_CHEST_X,SPINE_CHEST_Y,...,RIGHT_HIP_X,RIGHT_HIP_Y,LEFT_KNEE_X,LEFT_KNEE_Y,RIGHT_KNEE_X,RIGHT_KNEE_Y,LEFT_ANKLE_X,LEFT_ANKLE_Y,RIGHT_ANKLE_X,RIGHT_ANKLE_Y
0,1.0,0.0,1240.2129,205.73492,2.0,1244.6250,171.83230,2.0,1248.4340,143.70065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.0,1207.6191,203.61984,2.0,1208.2119,170.29300,2.0,1208.8193,142.77872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,0.0,1173.9341,219.31757,2.0,1175.3003,183.67642,2.0,1176.3620,154.28757,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,0.0,1139.3589,227.94061,2.0,1143.1482,191.05634,2.0,1146.3043,160.38876,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,0.0,1102.6255,232.28525,2.0,1105.8740,194.35587,2.0,1108.8252,162.85236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,1.0,0.0,1205.6267,237.32578,2.0,1207.1140,194.01947,2.0,1210.7063,156.89807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,1.0,0.0,1228.3091,240.78043,2.0,1233.0073,201.34854,2.0,1238.3262,167.99554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,1.0,0.0,1257.5059,238.24185,2.0,1267.4167,202.11438,2.0,1276.3973,171.71112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,0.0,0.0,0.0000,0.00000,0.0,0.0000,0.00000,0.0,0.0000,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# 데이터와 레이블을 추출합니다
label_data = full_data['LABEL']
data_without_label = full_data.drop(columns=[col for col in full_data.columns if "CONFIDENCE_LEVEL" in col or "spin" in col.lower()])
data_without_label = data_without_label.drop(['ID', 'TIMESTAMP', 'FRAME_NUM', 'LABEL', 'PELVIS_X', 'PELVIS_Y', 'NECK_X', 'NECK_Y'], axis=1)

In [12]:
data_without_label

,NUMOFBODIES,SHOULDER_LEFT_X,SHOULDER_LEFT_Y,ELBOW_LEFT_X,ELBOW_LEFT_Y,WRIST_LEFT_X,WRIST_LEFT_Y,SHOULDER_RIGHT_X,SHOULDER_RIGHT_Y,ELBOW_RIGHT_X,...,RIGHT_HIP_X,RIGHT_HIP_Y,LEFT_KNEE_X,LEFT_KNEE_Y,RIGHT_KNEE_X,RIGHT_KNEE_Y,LEFT_ANKLE_X,LEFT_ANKLE_Y,RIGHT_ANKLE_X,RIGHT_ANKLE_Y
0,1.0,1278.1079,114.825320,1259.6832,171.31137,1241.9669,215.73785,1226.9725,114.992830,1220.1800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1240.2539,110.277860,1237.1813,165.81174,1225.5015,210.13467,1183.5850,116.050810,1170.8182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1212.6339,126.382416,1213.9589,184.40631,1210.9862,231.56741,1152.1658,122.803440,1141.2323,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1181.0144,127.760376,1186.6545,187.13950,1178.0974,236.86618,1122.3578,126.987976,1102.6381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1146.0264,132.623500,1154.9545,188.39767,1145.7614,237.44797,1081.5137,130.622990,1064.9300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2821855,0.0,0.0000,0.000000,0.0000,0.00000,0.0000,0.00000,0.0000,0.000000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2821856,0.0,0.0000,0.000000,0.0000,0.00000,0.0000,0.00000,0.0000,0.000000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2821857,0.0,0.0000,0.000000,0.0000,0.00000,0.0000,0.00000,0.0000,0.000000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2821858,0.0,0.0000,0.000000,0.0000,0.00000,0.0000,0.00000,0.0000,0.000000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
data_without_label.dtypes

NUMOFBODIES        float64
SHOULDER_LEFT_X    float64
SHOULDER_LEFT_Y    float64
ELBOW_LEFT_X       float64
ELBOW_LEFT_Y       float64
                    ...   
RIGHT_KNEE_Y       float64
LEFT_ANKLE_X       float64
LEFT_ANKLE_Y       float64
RIGHT_ANKLE_X      float64
RIGHT_ANKLE_Y      float64
Length: 62, dtype: object

In [16]:
sequence_start_indices = full_data.index[full_data['index_num'] == 0].tolist()
sequence_lengths = [sequence_start_indices[i] - sequence_start_indices[i-1] for i in range(1, len(sequence_start_indices))]
sequence_lengths.insert(0, sequence_start_indices[0])

# Min sequence length
min_sequence_length = min(sequence_lengths)

In [17]:
min_sequence_length = 610

In [18]:
X_sequences = []
y_sequences = []

# Iterating over the start indices to create sequences
for i in range(len(sequence_start_indices) - 1):
    start_index = sequence_start_indices[i]
    end_index = sequence_start_indices[i + 1]

    # Ensure the sequence has the expected length before appending
    if end_index - start_index == min_sequence_length:
        X_sequence = data_without_label.iloc[start_index:end_index].values
        y_sequence = label_data.iloc[start_index:end_index].values
        X_sequences.append(X_sequence)
        y_sequences.append(y_sequence[-1])  # Taking the label of the last frame for the sequence

# Converting lists to numpy arrays
X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

X_sequences.shape, y_sequences.shape

((4625, 610, 62), (4625,))

In [28]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X_sequences, y_sequences, test_size=0.3, random_state=48)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=48)

In [29]:
X_train.shape

(4238, 610, 28)

In [30]:
## 오버샘플링 (SMOTE)
#  - 2차원 이하 차원만 가능

# 학습데이터 차원 낮추기 (3차원 -> 2차원)
# X_train.shape -> (4303, 610, 28)
X_train = X_train.reshape(4238, 610 * 28)

# SMOTE를 적용할 때는 반드시 !!학습 데이터!!만 오버샘플링해야 함
smote = SMOTE(random_state=48)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

X_smote.shape, y_smote.shape

((7428, 17080), (7428,))

In [31]:
# 원래대로 차원 늘리기 (2차원 -> 3차원)
# X_smote의 변경된 데이터 개수로 바꿔주기
# - 행위마다 오버샘플링 시 데이터 개수가 다름
X_smote = X_smote.reshape(7428, 610, 28)

In [32]:
X_train.shape, X_smote.shape

((4238, 17080), (7428, 610, 28))

In [33]:
print(X_train.shape, y_train.shape)

(4238, 17080) (4238,)


In [34]:
print(X_temp.shape, y_temp.shape)

(1817, 610, 28) (1817,)


In [35]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_val = X_val.astype(np.float32)
y_val = y_val.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

X_smote = X_smote.astype(np.float32)
y_smote = y_smote.astype(np.float32)

In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(X_smote.shape[1], X_smote.shape[2])),
    tf.keras.layers.LSTM(64, return_sequences=True),
    #tf.keras.layers.Dropout(0.3),
    #tf.keras.layers.LSTM(32),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 훈련 데이터에 대한 샘플 가중치 마스크 생성
# 패딩이 아닌 값은 1, 패딩된 값은 0
sample_weight_train = np.where(X_smote != 0, 1, 0)
sample_weight_train = sample_weight_train.max(axis=-1)

# 각 시퀀스에 대한 평균 샘플 가중치 계산
sample_weight_train_avg = sample_weight_train.mean(axis=1)

In [37]:
# !!!!!행위마다 모델 이름 변경하기!!!!!
model_file_path = "./model/smoke.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(model_file_path, save_best_only = True)
# - 자동 훈련 멈추기 함수 사용 : 추가 훈련 epoch 3회, 가중치 업데이트
# - patience : 검증 손실의 최솟값에 도달한 후 중단하기 전에 기다려야 하는 epoch 수
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3,
                                                  restore_best_weights=True)

# 모델 훈련
history = model.fit(X_smote, y_smote, validation_data=(X_val, y_val), epochs=500,
                    batch_size=128, sample_weight=sample_weight_train_avg,
                    callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/500
59/59 [==============================] - 315s 5s/step - loss: 0.5961 - accuracy: 0.8343 - val_loss: 0.1279 - val_accuracy: 0.9512
Epoch 2/500
59/59 [==============================] - 295s 5s/step - loss: 0.0585 - accuracy: 0.9789 - val_loss: 0.1420 - val_accuracy: 0.9552
Epoch 3/500
59/59 [==============================] - 386s 7s/step - loss: 0.0388 - accuracy: 0.9856 - val_loss: 0.0638 - val_accuracy: 0.9772
Epoch 4/500
 8/59 [===>..........................] - ETA: 5:34 - loss: 0.0103 - accuracy: 0.9971

KeyboardInterrupt: 

In [ ]:
# 모델 불러오기
from keras.models import load_model
model1 = load_model('./model/smoke.h5')

In [ ]:
X_sequences, y_sequences
X_val = X_sequences.astype(np.float32)
y_val = y_sequences.astype(np.float32)

In [ ]:
# 유니크한 종속변수들을 얻습니다.
unique_targets = np.unique(y_val)

# 테스트 데이터에 대한 예측을 한 번만 수행
test_preds = model1.predict(X_val)
test_pred_idx = [np.argmax(pred) for pred in test_preds]

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# 실제 레이블과 예측 레이블을 사용하여 Confusion Matrix를 계산합니다.
conf_matrix = confusion_matrix(y_val, test_pred_idx)

# F1 Score를 계산합니다.
f1 = f1_score(y_val, test_pred_idx, average='weighted')

# Confusion Matrix를 시각화합니다.
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
# 모델에 맞게 타이틀 변경하기
plt.title('smoke')
plt.show()

# 결과를 출력합니다.
print("F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_val, test_pred_idx))